In [56]:
import requests
from dotenv import load_dotenv
import json
import os
import urllib.parse as urlencoder

from google.cloud import aiplatform
import vertexai
from langchain_google_vertexai import VertexAI

### NEWS API TEST

In [46]:
load_dotenv()

NEWS_API_KEY = os.environ.get('NEWS_API_KEY')
NEWS_API_URL = "https://newsapi.org/v2/everything?q={keyword}&from={from_date}&sortBy=popularity&country=ko&apiKey={API_KEY}"
NEWS_API_HEADLINE_URL = "https://newsapi.org/v2/top-headlines?category=business&country=kr&apiKey={API_KEY}"

In [39]:
keyword = urlencoder.quote("SeAH")
from_date = '2024-02-12'

In [47]:
response = requests.get(NEWS_API_HEADLINE_URL.format(API_KEY=NEWS_API_KEY))

In [50]:
import codecs

print(codecs.decode(json.dumps(response.json(), indent=2), 'unicode_escape'))

{
  "status": "ok",
  "totalResults": 70,
  "articles": [
    {
      "source": {
        "id": null,
        "name": "Donga.com"
      },
      "author": null,
      "title": "[단독]공사비 치솟는 건설 현장… 노조 월례비도 편법 부활 - 동아일보",
      "description": "“원자재값 인상 때문에 공사비가 올라간다고 하잖아요? 근데 사실은 노조 때문에 오르는 인건비 영향이 못지않습니다.” 경기 의정부시 한 건축 현장의 하청업체 관계자는 11일 이렇게…",
      "url": "https://www.donga.com/news/Economy/article/all/20240312/123924867/1",
      "urlToImage": "https://dimg1.donga.com/wps/NEWS/IMAGE/2024/03/12/123924859.1.jpg",
      "publishedAt": "2024-03-11T18:00:00Z",
      "content": "10 , 60 
 ? .11 . , . . , . . 15 . .
. . .
 . (OT) . OT . 10 OT 60 . 10 500 . , OT ..
 A. 1 () () 4 (OT) 2490 . 1 600 . OT 10 . 1 60 , 1114 .1 110140 OT 6 500 . . A 45 OT , .
11 . OT . . 50 . 10… [+135 chars]"
    },
    {
      "source": {
        "id": null,
        "name": "YouTube"
      },
      "author": null,
      "title": "조용병 은행연합회장 "홍콩 ELS 손실 사태 유감" / YTN - YTN",
      "description": "조용병 은행연합회장은 홍콩 H지수 대규

In [54]:
VERTEXAI_LOCATION = os.getenv("VERTEXAI_LOCATION")
vertexai.init(location=VERTEXAI_LOCATION)

In [57]:
llm_vertex = VertexAI(
    #model_name="text-bison@latest",
    model_name="text-bison-32k@002",
    max_output_tokens=8000,
    temperature=0,
    top_p=0.8,
    top_k=40,
)

llm = llm_vertex

In [59]:
import json
import ast

def parse_json_response(llm_json_response) -> any:
  #print('llm response:'+ response)
  start_char = '['
  end_char = ']'
  if llm_json_response.find('[') == -1 or max(0,llm_json_response.find('{')) < llm_json_response.find('[') :
    start_char = '{'
    end_char = '}'
  start_index = llm_json_response.find(start_char)
  end_index = llm_json_response.rfind(end_char)
  json_data = llm_json_response[start_index:end_index+1]
  parsed_json = json.loads(json_data)
  return parsed_json

def parse_python_object(llm_python_object) -> any:
  print('llm response:'+ llm_python_object)
  if llm_python_object.find('{') == -1:
    start_char = '['
    end_char = ']'
  elif llm_python_object.find('[') == -1 or llm_python_object.find('{') < llm_python_object.find('[') :
    start_char = '{'
    end_char = '}'
  start_index = llm_python_object.find(start_char)
  end_index = llm_python_object.rfind(end_char)
  object_data = llm_python_object[start_index:end_index+1]
  print(object_data)
  parsed_object = ast.literal_eval(object_data)
  return parsed_object

In [64]:
def get_summary_from_latest_news(headline_news):
  titles = [article['title'] + "\n" + article['description'] for article in headline_news['articles'][0:40]]

  prompt_template = """You are a steel industry analyst. Please summarize the following news articles which are related with steel industry in Korean 3~5 phrases.

  news: 
  {news}

  summary:

  """
  llm_response = llm_vertex.invoke(prompt_template.format(news="\n".join(titles)))
  print(llm_response)
  return llm_response



get_summary_from_latest_news(response.json())

 - 건설 현장 노조 월례비 편법 부활로 인건비 상승
- 은행연합회장, 홍콩 ELS 손실 사태에 유감 표명
- 서울대 등 대학 이공계 연구실, R&D 예산 삭감으로 마비
- 비트코인 가격 사상 최고치 경신
- 정용진 신세계그룹 부회장, 법적 책임 질 등기이사 미담임 논란
- 엔비디아 주가 폭락, 테슬라 순매수 지속
- 엔비디아, 저작권 보호 작품 허락 없이 사용해 집단소송 직면
- 미국 마트 가방 가격 급등
- 하루인베스트, 자본잠식 벗어나려 가상자산 빼돌려
- 엔비디아 급락 여파에 미국 물가 지표 경계 심리 확산, 코스피 하락 전환
- 임종윤 한미약품 대표, OCI와 통합에 대해 "제약산업 이해한다면 내릴 수 없는 판단"
- 강호동 농협중앙회장 취임, "농·축협 중심의 든든한 농협 만들 것"
- 현대차·기아, 초급속 충전기 전국에 설치
- 삼성 파운드리, 미국 반도체 보조금 확정 발표로 다시 날개 단다
- 개미들, 1200원에서 400원으로 폭락한 주식에 속수무책
- 금감원, 2개월 만에 4급 직원 4명 퇴사로 MZ세대 이탈 가속


' - 건설 현장 노조 월례비 편법 부활로 인건비 상승\n- 은행연합회장, 홍콩 ELS 손실 사태에 유감 표명\n- 서울대 등 대학 이공계 연구실, R&D 예산 삭감으로 마비\n- 비트코인 가격 사상 최고치 경신\n- 정용진 신세계그룹 부회장, 법적 책임 질 등기이사 미담임 논란\n- 엔비디아 주가 폭락, 테슬라 순매수 지속\n- 엔비디아, 저작권 보호 작품 허락 없이 사용해 집단소송 직면\n- 미국 마트 가방 가격 급등\n- 하루인베스트, 자본잠식 벗어나려 가상자산 빼돌려\n- 엔비디아 급락 여파에 미국 물가 지표 경계 심리 확산, 코스피 하락 전환\n- 임종윤 한미약품 대표, OCI와 통합에 대해 "제약산업 이해한다면 내릴 수 없는 판단"\n- 강호동 농협중앙회장 취임, "농·축협 중심의 든든한 농협 만들 것"\n- 현대차·기아, 초급속 충전기 전국에 설치\n- 삼성 파운드리, 미국 반도체 보조금 확정 발표로 다시 날개 단다\n- 개미들, 1200원에서 400원으로 폭락한 주식에 속수무책\n- 금감원, 2개월 만에 4급 직원 4명 퇴사로 MZ세대 이탈 가속'